# Regression model in keras part D

Lets import some libraries

In [9]:
import keras
import pandas as pd
import numpy as np

Reading data concrete_data.csv file using pandas and observe some of its sample

In [10]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_0b07da65cd15422fa9b518e4935d324e = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='vXh_dgcbfOt_gngJOlUZ1wBxGZkardKL9hc3GUyIucU1',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_0b07da65cd15422fa9b518e4935d324e.get_object(Bucket='bigdata-donotdelete-pr-qzh61zisjysu4c',Key='concrete_data.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()


,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


Let's seperate feature and target column and view the shape

In [11]:
print(df.shape)
X=df[["Cement",'Blast Furnace Slag','Fly Ash','Water','Superplasticizer','Coarse Aggregate','Fine Aggregate','Age']]
y=df[['Strength']]
print(X.shape)
print(y.shape)



(1030, 9)
(1030, 8)
(1030, 1)


Now normalize the data using preprocessing library from scikit-learn

In [12]:
from sklearn import preprocessing
X_norm=preprocessing.normalize(X.values)
X_norm1=pd.DataFrame(X_norm, columns=["Cement",'Blast Furnace Slag','Fly Ash','Water','Superplasticizer','Coarse Aggregate','Fine Aggregate','Age'])
X_norm1.head()


,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,0.396244,0.000000,0.0,0.118873,0.001834,0.763138,0.496039,0.020546
1,0.392934,0.000000,0.0,0.117880,0.001819,0.767677,0.491895,0.020374
2,0.273572,0.117245,0.0,0.187592,0.000000,0.766823,0.488726,0.222148
3,0.268151,0.114922,0.0,0.183875,0.000000,0.751630,0.479043,0.294361
4,0.145536,0.097024,0.0,0.140700,0.000000,0.716983,0.604936,0.263812


Split train and test data by 70:30 using train_test_split from scikit-learn

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X_norm,y,test_size=0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(721, 8)
(309, 8)
(721, 1)
(309, 1)


Let's build the model

In [14]:
from keras.models import Sequential
from keras.layers import Dense
model=Sequential()

In [15]:
n_cols=X_train.shape[1]
print(n_cols)
model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

8


In [16]:
model.compile(optimizer="adam", loss="mean_squared_error")
model.fit(X_train, y_train, epochs=50, verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
721/721 [==============================] - 1s 2ms/step - loss: 1581.6222
Epoch 2/50
721/721 [==============================] - 0s 415us/step - loss: 1565.2740
Epoch 3/50
721/721 [==============================] - 0s 461us/step - loss: 1540.4962
Epoch 4/50
721/721 [==============================] - 0s 399us/step - loss: 1501.4134
Epoch 5/50
721/721 [==============================] - 0s 446us/step - loss: 1441.6328
Epoch 6/50
721/721 [==============================] - 0s 446us/step - loss: 1351.5579
Epoch 7/50
721/721 [==============================] - 0s 416us/step - loss: 1225.2209
Epoch 8/50
721/721 [==============================] - 0s 383us/step - loss: 1060.6686
Epoch 9/50
721/721 [==============================] - 0s 542us/step - loss: 866.5849
Epoch 10/50
721/721 [==============================] - 0s 414us/step - loss: 663.4597
Epoch 11/50
721/721 [==============================] - 0s 451us/step - loss: 482.0565
Epoch 12/

In [17]:
y_predict=model.predict(X_test)

Evaluation of the model by mean_squared_error from scikit-learn

In [18]:
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(y_predict, y_test)
mse

241.633466797868

Let's iterate the model for 50 iterations

In [19]:
iterate=50
mse_total=[]
for i in range (0, iterate):
    X_train, X_test, y_train, y_test=train_test_split(X_norm,y,test_size=0.3, random_state=i)
    #model=Sequential()
    #n_cols=X_train.shape[1]
    #model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    #model.add(Dense(1))
    model.compile(optimizer="adam", loss="mean_squared_error")
    model.fit(X_train, y_train, epochs=50, verbose=1)
    y_predict=model.predict(X_test)
    mse=mean_squared_error(y_predict, y_test)
    mse_total.append(mse)
print(mse_total)
    
    
    
    

Epoch 1/50
721/721 [==============================] - 1s 2ms/step - loss: 256.0138
Epoch 2/50
721/721 [==============================] - 0s 440us/step - loss: 254.3842
Epoch 3/50
721/721 [==============================] - 0s 411us/step - loss: 253.2815
Epoch 4/50
721/721 [==============================] - 0s 480us/step - loss: 252.8204
Epoch 5/50
721/721 [==============================] - 0s 497us/step - loss: 251.8593
Epoch 6/50
721/721 [==============================] - 1s 2ms/step - loss: 250.6405
Epoch 7/50
721/721 [==============================] - 0s 531us/step - loss: 249.8868
Epoch 8/50
721/721 [==============================] - 0s 434us/step - loss: 248.9694
Epoch 9/50
721/721 [==============================] - 0s 423us/step - loss: 248.1668
Epoch 10/50
721/721 [==============================] - 0s 461us/step - loss: 247.0298
Epoch 11/50
721/721 [==============================] - 0s 468us/step - loss: 246.9778
Epoch 12/50
721/721 [==============================] - 0s 438us/ste

Final Result

In [20]:
mean=np.mean(mse_total)
standard_deviation=np.std(mse_total)
print("Mean: %.2f" %mean)
print("standard deviation: %.2f" %standard_deviation)

Mean: 75.95
standard deviation: 35.01


Remarks: Mean of mean_squared error has been improved significantly than previous part B 

Part B: Mean: 132.71, standard deviation: 37.46

